##### Primero generamos y guardamos una clave de encriptación en un archivo para su uso posterior, y establecemos una conexión a una base de datos SQLite para crear una tabla Usuarios con columnas para ID, Nombre, Email, y DatosSensibles.

In [3]:
from cryptography.fernet import Fernet
import sqlite3
import os

# Generar una clave y guardarla en un archivo (esto una sola vez)
def generar_clave():
    key = Fernet.generate_key()
    with open("secret.key", "wb") as key_file:
        key_file.write(key)

# Leer la clave desde el archivo
def cargar_clave():
    return open("secret.key", "rb").read()

conn = sqlite3.connect('seguridad.db')
cursor = conn.cursor()

# Crear una tabla en la base de datos
def crear_tabla():
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Usuarios (
        ID INTEGER PRIMARY KEY,
        Nombre TEXT NOT NULL,
        Email TEXT UNIQUE,
        DatosSensibles BLOB
    );
    ''')

##### Luego creamos una funcion que encripta datos sensibles usando la clave de encriptación cargada previamente, y los guarda en la base de datos SQLite, intentando hasta 5 veces si ocurre un error de operación en la base de datos.

In [4]:
def encriptar_y_guardar_datos(nombre, email, datos_sensibles):
    key = cargar_clave()
    f = Fernet(key)
    datos_encriptados = f.encrypt(datos_sensibles.encode())
    for _ in range(5):  # Reintentar hasta 5 veces
        try:
            conn = sqlite3.connect('seguridad.db')
            cursor = conn.cursor()
            cursor.execute("INSERT INTO Usuarios (Nombre, Email, DatosSensibles) VALUES (?, ?, ?)",
                           (nombre, email, datos_encriptados))
            conn.commit()
            print(f"Datos para {nombre} encriptados y guardados correctamente.")
            conn.close()
            break
        except sqlite3.OperationalError as e:
            print(f"Error: {e}, reintentando...")

##### Luego creamos otra funcion que intenta hasta 5 veces recuperar y desencriptar datos sensibles de un usuario, utilizando una clave de encriptación cargada previamente y manejando errores operacionales de la base de datos.

In [6]:
def recuperar_y_desencriptar_datos(email):
    key = cargar_clave()
    f = Fernet(key)
    for _ in range(5):  # Reintentar hasta 5 veces
        try:
            conn = sqlite3.connect('seguridad.db')
            cursor = conn.cursor()
            cursor.execute("SELECT Nombre, Email, DatosSensibles FROM Usuarios WHERE Email = ?", (email,))
            usuario = cursor.fetchone()
            if usuario is None:
                print(f"No se encontraron datos para el email: {email}")
                return None, None, None
            datos_desencriptados = f.decrypt(usuario[2]).decode()
            return usuario[0], usuario[1], datos_desencriptados
        except sqlite3.OperationalError as e:
            print(f"Error: {e}, reintentando...")

Mostrar el flujo final de las funciones

##### Creamos una funcion para probar las creadas anteriormente, verificamos si existe una clave de encriptación y generamos una si es necesario, creamos una tabla en la base de datos SQLite, encriptamos y guardamos datos sensibles, luego recuperamos y desencriptamos esos datos, mostrando el flujo completo de encriptación y desencriptación para un usuario de ejemplo.

In [8]:
conn = sqlite3.connect('seguridad.db')
cursor = conn.cursor()
def mostrar_flujo():
    # Generar la clave si no existe
    if not os.path.exists("secret.key"):
        generar_clave()
    
    # Crear la tabla en la base de datos
    crear_tabla()

    # Datos de ejemplo
    nombre = "Tutu"
    email = "Tutu@Penguin.com"
    datos_sensibles = "Información muy confidencial"

    # Encriptar y guardar datos
    encriptar_y_guardar_datos(nombre, email, datos_sensibles)
    print(f"Datos encriptados y guardados para {nombre}.")

    # Recuperar y desencriptar datos
    nombre_recuperado, email_recuperado, datos_desencriptados = recuperar_y_desencriptar_datos(email)
    print(f"Nombre: {nombre_recuperado}")
    print(f"Email: {email_recuperado}")
    print(f"Datos Sensibles: {datos_desencriptados}")

# Ejecutar la demostración
mostrar_flujo()


Datos para Edu encriptados y guardados correctamente.
Datos encriptados y guardados para Edu.
Nombre: Edu
Email: Edu@Penguin.com
Datos Sensibles: Información muy confidencial


### Consulta normal sin desencriptar los datos

In [10]:
print("\nConsulta sin índice:")
for row in cursor.execute("SELECT Nombre, Email, DatosSensibles FROM Usuarios"):
    print(row)


Consulta sin índice:
('Gus', 'Gus@Penguin.com', b'gAAAAABmonbOORLBSwNIy_mf79MuHSqVPFDClzd-kW1STPRcBtyyDXb5kQeWuuFEPfpN1hbnAhi-APK7dwrH9f1n2ebcz00GEU33uWR62GjIaliPEhc39RY=')
('Edu', 'Edu@Penguin.com', b'gAAAAABmookpxARooj8Hbfo8Xyepa89gGXCiniFH6SwryvSRnHKHxTbT9zsP82DrfOCxVqjBTRCes9uxQThC1zentcwngssJKPtCPKvHjnoX_TuVWRqL9Zc=')


## Inyecciones SQL

Creamos una nueva tablas de Credenciales

In [11]:
# Crear la tabla Credenciales
cursor.execute('''
CREATE TABLE IF NOT EXISTS Credenciales (
    id INTEGER PRIMARY KEY,
    username TEXT NOT NULL,
    contrasena TEXT NOT NULL
);
''')

# Insertar datos de ejemplo en la tabla Usuarios
cursor.execute("INSERT OR IGNORE INTO Credenciales (username, contrasena) VALUES ('admin', 'admin123')")
cursor.execute("INSERT OR IGNORE INTO Credenciales (username, contrasena) VALUES ('user', 'user123')")
cursor.execute("INSERT OR IGNORE INTO Credenciales (username, contrasena) VALUES ('test', 'test123')")

conn.commit()

## Ejemplo Inseguro
La siguiente función genera una consulta SQL directamente insertando los valores username y password en la consulta, sin usar consultas parametrizadas. Esto deja la aplicación vulnerable a inyecciones SQL. 

In [12]:
def login_vulnerable(username, password):
    query = f"SELECT * FROM Credenciales WHERE username = '{username}' AND contrasena = '{password}'" # 
    print("Consulta SQL generada:", query)  # Para fines educativos, mostramos la consulta generada
    cursor.execute(query)
    result = cursor.fetchall()  # Obtener todos los resultados
    conn.close()
    return result

#### Problemas de seguridad en la funcion anterior

- Las entradas del usuario no se sanitizan ni se validan antes de ser usadas en la consulta SQL.
- Al concatenar directamente las entradas de usuario en la consulta SQL, se permite que cualquier entrada maliciosa se ejecute como parte de la consulta SQL.

El atacante ingresa ' OR '1'='1 tanto en el campo de nombre de usuario como en el de contraseña.

##### La consulta se evalúa en dos partes:

- username = '' OR '1'='1': Esta parte es siempre verdadera porque '1'='1' es una condición lógica que siempre se cumple.
- AND contrasena = '' OR '1'='1': Esta parte también es siempre verdadera por la misma razón.

Debido a que ambas condiciones son verdaderas, la consulta completa devuelve todas las filas de la tabla Credenciales.

##### Como resultado, el atacante obtiene acceso a la base de datos sin necesidad de conocer el nombre de usuario o la contraseña real. La condición '1'='1' hace que la consulta sea siempre verdadera, permitiendo el acceso no autorizado.

In [13]:
# Ejemplo de uso
# Supongamos que el atacante ingresa lo siguiente:
username = "' OR '1'='1"
password = "' OR '1'='1"

login_vulnerable(username, password)


Consulta SQL generada: SELECT * FROM Credenciales WHERE username = '' OR '1'='1' AND contrasena = '' OR '1'='1'


[(1, 'admin', 'admin123'),
 (2, 'user', 'user123'),
 (3, 'test', 'test123'),
 (4, 'admin', 'admin123'),
 (5, 'user', 'user123'),
 (6, 'test', 'test123')]

## Ejemplo Seguro

Las consultas parametrizadas evitan que el código SQL malicioso sea ejecutado, protegiendo la base de datos contra ataques de inyección SQL. Al usar parámetros (?), SQLite se encarga de sanitizar los valores de entrada, asegurando que se traten como datos y no como parte del código SQL.

La separación clara entre el código SQL y los datos de entrada asegura que la consulta se ejecute de manera segura, independientemente de los valores proporcionados por el usuario.

In [14]:
# Función segura usando consultas parametrizadas
def login_seguro(username, password):
    conn = sqlite3.connect('seguridad.db')
    cursor = conn.cursor()
    query = "SELECT * FROM Credenciales WHERE username = ? AND contrasena = ?"
    cursor.execute(query, (username, password))
    result = cursor.fetchall()
    conn.close()
    return result

# Ejemplo de uso seguro
username = input("Nombre de usuario: ")
password = input("Contraseña: ")
users = login_seguro(username, password)
print("Resultados de la consulta:")
for user in users:
    print(user)

Resultados de la consulta:


### Volver a probar en el input:
- username = "' OR '1'='1"
- password = "' OR '1'='1"